In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def softmax_cross_entropy_loss(z,t):
   probs = np.exp(z - np.max(z, axis=1, keepdims=True))
   probs /= np.sum(probs, axis=1, keepdims=True)
   N = z.shape[0]
   print "Num in Batch",N
   loss = -np.sum(np.log(probs[np.arange(N), t])) / N
   dz = probs.copy()
   dz[np.arange(N), t] -= 1
   dz /= N
   return loss, dz

In [3]:
def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [4]:
num_inputs = 3
num_classes = 3
z = 0.001 * np.random.randn(num_inputs, num_classes)
y = np.random.randint(num_classes, size=num_inputs)
dz_num = np.zeros((num_inputs,num_classes))
small_e = 0.0001
print z
for ii in range(num_inputs):
    for z_ in range(num_classes):
        z_working = z.copy()
        z_working[ii,z_] += small_e
        cost_num1,_ = softmax_cross_entropy_loss(z_working,y)
        z_working[ii,z_] -= 2*small_e
        cost_num2,_ = softmax_cross_entropy_loss(z_working,y)
        z_working[ii,z_] += small_e
        dz_num[ii,z_] = (cost_num1 - cost_num2)/(2*small_e)
    
print "numerical solution:",dz_num
_, dz_cal = softmax_cross_entropy_loss(z,y)
print "cal solution:",dz_cal
print "difference:",rel_error(dz_cal,dz_num)

[[ 0.00054486 -0.00207678  0.00142793]
 [-0.00204264 -0.00016088  0.0001374 ]
 [-0.00045431  0.00156296 -0.00169609]]
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
numerical solution: [[ 0.1111754  -0.22244902  0.11127362]
 [ 0.11096072  0.11116972 -0.22213045]
 [ 0.11108229  0.1113066  -0.22238889]]
Num in Batch 3
cal solution: [[ 0.1111754  -0.22244902  0.11127362]
 [ 0.11096072  0.11116972 -0.22213045]
 [ 0.11108229  0.1113066  -0.22238889]]
difference: 1.90382564921e-10


In [5]:
def eval_numerical_gradient_array(f, x, df, h=1e-5):
  """
  Evaluate a numeric gradient for a function that accepts a numpy
  array and returns a numpy array.
  """
  grad = np.zeros_like(x)
  it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
  while not it.finished:
    ix = it.multi_index
    
    oldval = x[ix]
    x[ix] = oldval + h
    pos = f(x)
    x[ix] = oldval - h
    neg = f(x)
    x[ix] = oldval
    
    grad[ix] = np.sum((pos - neg) * df) / (2 * h)
    it.iternext()
   
  return grad

In [6]:
def eval_numerical_gradient(f, x, verbose=True, h=0.00001):
  """ 
  a naive implementation of numerical gradient of f at x 
  - f should be a function that takes a single argument
  - x is the point (numpy array) to evaluate the gradient at
  """ 

  fx = f(x) # evaluate function value at original point
  grad = np.zeros_like(x)
  # iterate over all indexes in x
  it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
  while not it.finished:

    # evaluate function at x+h
    ix = it.multi_index
    oldval = x[ix]
    x[ix] = oldval + h # increment by h
    fxph = f(x) # evalute f(x + h)
    x[ix] = oldval - h
    fxmh = f(x) # evaluate f(x - h)
    x[ix] = oldval # restore

    # compute the partial derivative with centered formula
    grad[ix] = (fxph - fxmh) / (2 * h) # the slope
    if verbose:
      print ix, grad[ix]
    it.iternext() # step to next dimension

  return grad

In [7]:
num_inputs = 3
num_classes = 3
x = 0.001 * np.random.randn(num_inputs, num_classes)
y = np.random.randint(num_classes, size=num_inputs)

dx_num = eval_numerical_gradient(lambda x: softmax_cross_entropy_loss(x, y)[0], x, verbose=False)
loss, dx = softmax_cross_entropy_loss(x, y)
print dx.shape, dx_num.shape
print dx, dx_num
# Test softmax_loss function. Loss should be 2.3 and dx error should be 1e-8
print '\nTesting softmax_loss:'
print 'loss: ', loss
print 'dx error: ', rel_error(dx_num, dx)

Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
Num in Batch 3
(3, 3) (3, 3)
[[ 0.11116081 -0.22216247  0.11100166]
 [ 0.11110319  0.11111676 -0.22221995]
 [ 0.11115648  0.11118839 -0.22234488]] [[ 0.11116081 -0.22216247  0.11100166]
 [ 0.11110319  0.11111676 -0.22221995]
 [ 0.11115648  0.11118839 -0.22234488]]

Testing softmax_loss:
loss:  1.0987944285
dx error:  5.53497577096e-11


In [8]:
def affine_forward(x, w, b):
  """
  Computes the forward pass for an affine (fully-connected) layer.

  The input x has shape (N, d_1, ..., d_k) where x[i] is the ith input.
  We multiply this against a weight matrix of shape (D, M) where
  D = \prod_i d_i

  Inputs:
  x - Input data, of shape (N, d_1, ..., d_k)
  w - Weights, of shape (D, M)
  b - Biases, of shape (M,)
  
  Returns a tuple of:
  - out: output, of shape (N, M)
  - cache: (x, w, b)
  """
  out = None
  #############################################################################
  # TODO: Implement the affine forward pass. Store the result in out. You     #
  # will need to reshape the input into rows.                                 #
  #############################################################################
  N = x.shape[0]
  D = np.prod(x.shape[1:])
  x2 = np.reshape(x, (N, D))
  out = np.dot(x2, w) + b
  #############################################################################
  #                             END OF YOUR CODE                              #
  #############################################################################
  cache = (x, w, b)
  return out, cache


In [21]:
def affine_backward(dout, cache):
  """
  Computes the backward pass for an affine layer.

  Inputs:
  - dout: Upstream derivative, of shape (N, M)
  - cache: Tuple of:
    - x: Input data, of shape (N, d_1, ... d_k)
    - w: Weights, of shape (D, M)

  Returns a tuple of:
  - dx: Gradient with respect to x, of shape (N, d1, ..., d_k)
  - dw: Gradient with respect to w, of shape (D, M)
  - db: Gradient with respect to b, of shape (M,)
  """
  x, w, b = cache
  dx, dw, db = None, None, None
  #############################################################################
  # TODO: Implement the affine backward pass.                                 #
  #############################################################################
  N = x.shape[0]
  D = x.shape[1]
 

  dx = np.dot(dout, w.T) # N x D
  dw = np.dot(x.T, dout) # D x M
  db = np.dot(dout.T, np.ones(N)) # M x 1
  print "dout shape,db shape",dout.shape,db.shape
  print "dout",dout
  print "db", db

  #############################################################################
  #                             END OF YOUR CODE                              #
  #############################################################################
  return dx, dw, db

In [24]:
x = np.random.randn(1, 3)
w = np.random.randn(3, 2)
b = np.random.randn(2)
dout = np.random.randn(1, 2)

dx_num = eval_numerical_gradient_array(lambda x: affine_forward(x, w, b)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: affine_forward(x, w, b)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: affine_forward(x, w, b)[0], b, dout)

_, cache = affine_forward(x, w, b)
dx, dw, db = affine_backward(dout, cache)

print dx_num, dx
# The error should be less than 1e-10
print 'Testing affine_backward function:'
print 'dx error: ', rel_error(dx_num, dx)
print 'dw error: ', rel_error(dw_num, dw)
print 'db error: ', rel_error(db_num, db)

dout shape,db shape (1, 2) (2,)
dout [[-0.59886226 -1.39554967]]
db [-0.59886226 -1.39554967]
[[-1.62598403  0.30624973  0.79320264]] [[-1.62598403  0.30624973  0.79320264]]
Testing affine_backward function:
dx error:  1.15962670694e-11
dw error:  1.0458929699e-11
db error:  3.27554041641e-12


In [11]:
model_hyper = {}
model_hyper['num_classes'] = 3
model_hyper['input_dim']=4
model_hyper['num_hidden_layers']=2
model_hyper['learnrate']=0.0003
layer_size = [None] * model_hyper['num_hidden_layers']
layer_size[0] = 5
layer_size[1] = 4
#layer_size[2] = 5
#layer_size[3] = 6
model_hyper['layer_sizes'] = layer_size    

In [12]:
print model_hyper['layer_sizes'][0]

5


In [13]:
class NeuralNet_SoftmaxCross:
    #class variables
    # num_classes  is the size of the last affine activation
    # input_dim   is the size of input dimension
    # num_hidden_layers is how many hidden layers
    ## (constructor ) Dict model
    ## (constructor) num_layers is how many totol layers = hidden_layers + 2(Softmax cross entropy loss layer)
    ## (Forward) Dict input cache
    ## (Backward) Dict gradient cache 
    #class def
    # constructor to construct the neural network, given the hyper parameter
    # Forward
    # Backward
    # applyGrad
    def __init__(self,model_hyper):
        num_classes = model_hyper['num_classes']
        input_dim  = model_hyper['input_dim']
        self.num_hidden_layers = model_hyper['num_hidden_layers']
        self.num_layers = self.num_hidden_layers + 1
        layer_list = [None] * self.num_hidden_layers
        cur_layer_size = input_dim
        self.learnrate = model_hyper['learnrate']
        self.NN_model = []
        self.NN_grad = []
        self.NN_num_grad = []
        for hid_layer_iter in range(self.num_hidden_layers):  
            layer_model = {}
            next_layer_size = model_hyper['layer_sizes'][hid_layer_iter]
            layer_model['W'] = 0.9 * np.random.randn(cur_layer_size, next_layer_size)
            layer_model['b'] =0.9 * np.random.randn(next_layer_size)
            self.NN_model.append(layer_model)
            cur_layer_size = next_layer_size
        
        layer_model = {}   
        next_layer_size = num_classes
        layer_model['W'] = 0.9 * np.random.randn(cur_layer_size, next_layer_size)
        self.NN_model.append(layer_model)
        self.NN_cache=[]
        for i in self.NN_model:
            print i['W'].shape
            if 'b' in i:
                print i['b'].shape
            print "__________________"
            
    def affine_backward(self,dout, cache):
        x = cache['x']
        w = cache['w']
        b = cache['b']
        #print x,w,b
        grad_cache = {}
        dx, dw, db = None, None, None
        N = x.shape[0]
        D = np.prod(x.shape[1:])
        x2 = np.reshape(x, (N, D))

        dx2 = np.dot(dout, w.T) # N x D
        dw = np.dot(x2.T, dout) # D x M
        db = np.dot(dout.T, np.ones(N)) # M x 1

        dx = np.reshape(dx2, x.shape)
        grad_cache['dx'] = dx
        grad_cache['dw'] = dw
        #print grad_cache['dw']
        grad_cache['db'] = db
        return grad_cache
    
    def affine_backward2(self,dout, cache):
        x = cache['x']
        w = cache['w']

        grad_cache = {}
        dx, dw = None, None
        N = x.shape[0]
        D = np.prod(x.shape[1:])
        x2 = np.reshape(x, (N, D))

        dx2 = np.dot(dout, w.T) # N x D
        dw = np.dot(x2.T, dout) # D x M       

        dx = np.reshape(dx2, x.shape)
        grad_cache['dx'] = dx
        grad_cache['dw'] = dw
 
        return grad_cache  

    def affine_forward(self,x, w, b): 
        cache = {}
        out = None
        N = x.shape[0]
        D = np.prod(x.shape[1:])
        x2 = np.reshape(x, (N, D))
        out = np.dot(x2, w) + b
        cache['x'] = x
        cache['w'] = w
        cache['b'] = b
        return out, cache    
    
    def affine_forward2(self,x, w): 
        out = None
        cache = {}
        N = x.shape[0]
        D = np.prod(x.shape[1:])
        x2 = np.reshape(x, (N, D))
        out = np.dot(x2, w) 
        cache['x'] = x
        cache['w'] = w
        return out, cache  
    
    def softmax(self,z):
        probs = np.exp(z - np.max(z, axis=1, keepdims=True))
        probs /= np.sum(probs, axis=1, keepdims=True)
        return probs
    
    def softmax_cross_entropy_loss(self,z,t):
        probs = np.exp(z - np.max(z, axis=1, keepdims=True))
        probs /= np.sum(probs, axis=1, keepdims=True)
        N = z.shape[0]
        #print "Num in Batch",N
        loss = -np.sum(np.log(probs[np.arange(N), t])) / N
        dz = probs.copy()
        dz[np.arange(N), t] -= 1
        dz /= N
        return loss, dz
    
    def forward_train(self,x,y):
        activation = x
        #print activation

        for i in self.NN_model:
            if 'b' in i:
                activation, cache = self.affine_forward(activation, i['W'], i['b'])
                self.NN_cache.append(cache)
            else:
                activation, cache = self.affine_forward2(activation, i['W'])
                self.NN_cache.append(cache)
        #print y.shape, activation.shape        
        loss,dz = self.softmax_cross_entropy_loss(activation,y)
        #for i in self.NN_cache:
        #    print i['w'].shape,i['x'].shape
        #    if 'b' in i:
        #        print i['b'].shape
        #    print "__________________"   
        return loss,dz
    
    def forward_test(self,x):
        activation = x
        #print activation

        for i in self.NN_model:
            if 'b' in i:
                activation, cache = self.affine_forward(activation, i['W'], i['b'])
                self.NN_cache.append(cache)
            else:
                activation, cache = self.affine_forward2(activation, i['W'])
                self.NN_cache.append(cache)
 
        return self.softmax(activation)
    
    def backward_train(self,dz):
        backgrad = dz
        for i in reversed(self.NN_cache):
            if 'b' in i:
                grad_cache = self.affine_backward(backgrad,i)
            else:
                grad_cache = self.affine_backward2(backgrad,i)
            backgrad = grad_cache['dx']
                
            self.NN_grad.insert(0,grad_cache)
            #self.NN_grad.append(grad_cache)
    def Apply_UpdateW(self):
        for i in range(self.num_layers):
            delta_W = -self.learnrate * self.NN_grad[i]['dw'] 
            self.NN_model[i]['W'] += delta_W
            #print delta_W.shape
            if 'db' in self.NN_grad[i] :
                delta_b = -self.learnrate * self.NN_grad[i]['db'] 
                self.NN_model[i]['b'] += delta_b
        self.reset_cache()

    def reset_cache(self):
        del(self.NN_cache[:])
        del(self.NN_grad[:]) 
        del(self.NN_num_grad[:]) 
        
    def get_grad(self):
        return self.NN_grad

    def get_W(self):
        return self.NN_model
    def cal_num_grad(self,w,h,x,y):
        grad = np.zeros_like(w)
        #print "w.shape",w.shape
        # iterate over all indexes in w
        it = np.nditer(w, flags=['multi_index'], op_flags=['readwrite'])
        
        while not it.finished:
            
            # evaluate function at x+h
            ix = it.multi_index
            oldval = w[ix]
            w[ix] = oldval + h # increment by h
            fxph,_ = self.forward_train(x,y) # evalute f(x + h)
            w[ix] = oldval - h
            fxmh,_ = self.forward_train(x,y) # evaluate f(x - h)
            w[ix] = oldval # restore
            # compute the partial derivative with centered formula
            grad[ix] = (fxph - fxmh) / (2 * h) # the slope
            it.iternext()           
        return grad
    def get_numerical_gradient(self):
        return self.NN_num_grad
    def set_numerical_gradient(self,x,y, h=0.0001):        
        for i in self.NN_model:
            num_grad_cache = {}
            num_grad_cache['dw'] = self.cal_num_grad(i['W'],h,x,y)
            
            if 'b' in i:
                num_grad_cache['db'] = self.cal_num_grad(i['b'],h,x,y)
               
            self.NN_num_grad.append(num_grad_cache)
            
        return self.NN_num_grad
    
    def check_grad_numgrad(self,x,y, h=0.0001,verbose = True):       
        self.set_numerical_gradient(x,y, h)  
        if verbose:
            for i in range(self.num_hidden_layers+1):
                print i,"th layer"
                if (rel_error(self.NN_grad[i]['dw'],self.NN_num_grad[i]['dw'])<1e-7):
                    print 'grad W correct',rel_error(self.NN_grad[i]['dw'],self.NN_num_grad[i]['dw'])
                else:
                    print self.NN_grad[i]['dw'].flatten()
                    print self.NN_num_grad[i]['dw'].flatten()
                    print rel_error(self.NN_grad[i]['dw'],self.NN_num_grad[i]['dw'])
                    print np.linalg.norm(self.NN_grad[i]['dw'].flatten() - self.NN_num_grad[i]['dw'].flatten())**2
                    print '==========='
                if 'db' in self.NN_grad[i] :
                    if (rel_error(self.NN_grad[i]['db'],self.NN_num_grad[i]['db'])<1e-7):
                        print 'grad b correct',rel_error(self.NN_grad[i]['db'],self.NN_num_grad[i]['db'])
                    else:
                        print self.NN_grad[i]['db'].flatten()
                        print self.NN_num_grad[i]['db'].flatten()
                        print rel_error(self.NN_grad[i]['db'],self.NN_num_grad[i]['db'])
                        print '==========='

In [14]:
a_nn = NeuralNet_SoftmaxCross(model_hyper)
num_inputs = 20
x = np.random.randn(num_inputs, model_hyper['input_dim'])
y = np.random.randint(model_hyper['num_classes'] , size=num_inputs)
print 'xshape',x.shape,y.shape
print '----------'
loss,dz = a_nn.forward_train(x,y)
print 'dz shape',dz.shape
a_nn.backward_train(dz)
grad = a_nn.get_grad()
a_nn.check_grad_numgrad(x,y)
print x

(4, 5)
(5,)
__________________
(5, 4)
(4,)
__________________
(4, 3)
__________________
xshape (20, 4) (20,)
----------
dz shape (20, 3)
0 th layer
grad W correct 1.02173369137e-08
grad b correct 1.51276159762e-09
1 th layer
grad W correct 7.62230931198e-09
grad b correct 2.21444584809e-10
2 th layer
grad W correct 2.96072007179e-09
[[-0.12305731  0.5455433   0.10026333 -0.13769783]
 [-1.51625294 -0.48911137 -0.30968825  0.08711234]
 [-1.69001067 -0.05020491 -0.58611398  0.03753668]
 [ 1.34356701  0.43968883  0.31153392 -0.07761616]
 [ 0.62076137 -1.17471808 -0.01097314  0.97900831]
 [-0.73341151 -0.29708798  0.78584429 -0.20912736]
 [-0.18639996 -1.41529911  1.12674678 -0.03072758]
 [ 0.00529501  0.49247559  0.78091851 -1.54598954]
 [ 0.49856873  1.0731052   0.32024565  0.17388135]
 [ 0.08336361 -0.38045953  0.97753755 -1.32980644]
 [ 0.14593522 -0.72769933 -0.87635074 -0.79014845]
 [-0.67339986 -0.44354168 -1.39091139 -0.9233488 ]
 [ 1.43911065  0.04845329  1.62887104  1.0474886 ]
 [

In [15]:
num_grad = a_nn.get_numerical_gradient()

In [16]:
for i in a_nn.get_W():
    print i['W'].shape
for i in grad:
    print i['dw'].shape
for i in num_grad:
    print i['dw'].shape

(4, 5)
(5, 4)
(4, 3)
(4, 5)
(5, 4)
(4, 3)
(4, 5)
(5, 4)
(4, 3)


In [17]:
print '=====num grad ====='
for i in range(model_hyper['num_hidden_layers']+1):
    print i,"th layer:W error",rel_error(grad[i]['dw'],num_grad[i]['dw'])    
    if 'db' in grad[i] :
        print i,"th layer:b error",rel_error(grad[i]['db'],num_grad[i]['db'])
    print "__________________"  

print grad[0]['dw'].flatten()
print num_grad[0]['dw'].flatten()
print "__________________"  
print grad[0]['db'].flatten()
print num_grad[0]['db'].flatten()
print "__________________"  
print grad[1]['dw'].flatten()
print num_grad[1]['dw'].flatten()
    

=====num grad =====
0 th layer:W error 1.02173369137e-08
0 th layer:b error 1.51276159762e-09
__________________
1 th layer:W error 7.62230931198e-09
1 th layer:b error 2.21444584809e-10
__________________
2 th layer:W error 2.96072007179e-09
__________________
[-0.41000283 -0.28540375  0.46619475 -0.16778069 -0.37632714 -0.05368755
 -0.01730436  0.01876853 -0.02559528 -0.03066958 -0.04257347  0.14865206
 -0.32719475 -0.0496307   0.12624584  0.0790943   0.22236975 -0.44241507
  0.00214078  0.22774305]
[-0.41000283 -0.28540375  0.46619475 -0.16778069 -0.37632714 -0.05368755
 -0.01730436  0.01876853 -0.02559528 -0.03066958 -0.04257347  0.14865206
 -0.32719475 -0.0496307   0.12624584  0.0790943   0.22236975 -0.44241507
  0.00214078  0.22774305]
__________________
[ 0.12023819  0.13335452 -0.24132975  0.04023289  0.15640788]
[ 0.12023819  0.13335452 -0.24132975  0.04023289  0.15640788]
__________________
[ 0.50560426  0.18814561  0.1082048   0.51343814  0.0546177  -0.04127529
 -0.24148942 

In [18]:
a_nn.Apply_UpdateW()
loss,dz = a_nn.forward_train(x,y)
print 'dz shape',dz.shape
a_nn.backward_train(dz)
grad = a_nn.get_grad()
a_nn.check_grad_numgrad(x,y)
num_grad = a_nn.get_numerical_gradient()
print '=====num grad ====='
for i in range(model_hyper['num_hidden_layers']+1):
    print i,"th layer:W error",rel_error(grad[i]['dw'],num_grad[i]['dw'])    
    if 'db' in grad[i] :
        print i,"th layer:b error",rel_error(grad[i]['db'],num_grad[i]['db'])
    print "__________________"  

print grad[0]['dw'].flatten()
print num_grad[0]['dw'].flatten()
print "__________________"  
print grad[0]['db'].flatten()
print num_grad[0]['db'].flatten()
print "__________________"  
print grad[1]['dw'].flatten()
print num_grad[1]['dw'].flatten()

dz shape (20, 3)
0 th layer
grad W correct 1.02598224294e-08
grad b correct 1.51478982135e-09
1 th layer
grad W correct 7.62767730174e-09
grad b correct 2.30173316038e-10
2 th layer
grad W correct 2.97774009469e-09
=====num grad =====
0 th layer:W error 1.02598224294e-08
0 th layer:b error 1.51478982135e-09
__________________
1 th layer:W error 7.62767730174e-09
1 th layer:b error 2.30173316038e-10
__________________
2 th layer:W error 2.97774009469e-09
__________________
[-0.409682   -0.28486385  0.46554808 -0.16738152 -0.37605416 -0.05342145
 -0.01704218  0.01833596 -0.02550065 -0.03035999 -0.04283308  0.14805799
 -0.32614869 -0.05000645  0.12590155  0.0790132   0.22198065 -0.44184682
  0.00174978  0.22771232]
[-0.409682   -0.28486385  0.46554808 -0.16738152 -0.37605416 -0.05342145
 -0.01704218  0.01833596 -0.02550065 -0.03035999 -0.04283308  0.14805799
 -0.32614868 -0.05000645  0.12590155  0.0790132   0.22198065 -0.44184682
  0.00174978  0.22771232]
__________________
[ 0.11968912  

In [19]:
a_nn.Apply_UpdateW()
loss,dz = a_nn.forward_train(x,y)
print 'dz shape',dz.shape
a_nn.backward_train(dz)
grad = a_nn.get_grad()
a_nn.check_grad_numgrad(x,y)
num_grad = a_nn.get_numerical_gradient()
print '=====num grad ====='
for i in range(model_hyper['num_hidden_layers']+1):
    print i,"th layer:W error",rel_error(grad[i]['dw'],num_grad[i]['dw'])    
    if 'db' in grad[i] :
        print i,"th layer:b error",rel_error(grad[i]['db'],num_grad[i]['db'])
    print "__________________"  

print grad[0]['dw'].flatten()
print num_grad[0]['dw'].flatten()
print "__________________"  
print grad[0]['db'].flatten()
print num_grad[0]['db'].flatten()
print "__________________"  
print grad[1]['dw'].flatten()
print num_grad[1]['dw'].flatten()

dz shape (20, 3)
0 th layer
grad W correct 1.02279302907e-08
grad b correct 1.52243330733e-09
1 th layer
grad W correct 7.64741671447e-09
grad b correct 2.13102002447e-10
2 th layer
grad W correct 2.99196634753e-09
=====num grad =====
0 th layer:W error 1.02279302907e-08
0 th layer:b error 1.52243330733e-09
__________________
1 th layer:W error 7.64741671447e-09
1 th layer:b error 2.13102002447e-10
__________________
2 th layer:W error 2.99196634753e-09
__________________
[-0.40936019 -0.28432422  0.46490083 -0.16698311 -0.37577974 -0.05315603
 -0.01678078  0.01790413 -0.02540698 -0.03005066 -0.04309297  0.14746512
 -0.32510396 -0.05038082  0.1255565   0.07893167  0.22159193 -0.44127859
  0.00135949  0.2276806 ]
[-0.40936019 -0.28432422  0.46490083 -0.16698311 -0.37577974 -0.05315603
 -0.01678078  0.01790413 -0.02540698 -0.03005066 -0.04309297  0.14746512
 -0.32510396 -0.05038082  0.1255565   0.07893167  0.22159193 -0.44127859
  0.00135949  0.2276806 ]
__________________
[ 0.11914007  

In [20]:
#training/ testing data

DataXy_ = np.loadtxt(open("iris_prep.txt", "rb"), delimiter=",", skiprows=1)
print DataXy_.shape
rand_idx = np.random.permutation(149)
DataXy =  DataXy_[rand_idx,:]
print DataXy[0:4,:]
num_train = 100
num_test = 48
trainX = DataXy[0:num_train,:-1]
meanX = trainX.mean()
print trainX
trainX = 0.1*(trainX-meanX) 
trainY = DataXy[0:num_train,-1]
testX = 0.1*(DataXy[num_train:,:-1]-meanX) 
testY = DataXy[num_train:,-1]
print trainX

(149, 5)
[[ 5.   3.4  1.6  0.4  2. ]
 [ 7.9  3.8  6.4  2.   0. ]
 [ 6.   2.7  5.1  1.6  1. ]
 [ 5.4  3.9  1.7  0.4  2. ]]
[[ 5.   3.4  1.6  0.4]
 [ 7.9  3.8  6.4  2. ]
 [ 6.   2.7  5.1  1.6]
 [ 5.4  3.9  1.7  0.4]
 [ 4.9  3.1  1.5  0.1]
 [ 5.5  4.2  1.4  0.2]
 [ 6.9  3.1  5.1  2.3]
 [ 6.9  3.1  5.4  2.1]
 [ 4.8  3.4  1.6  0.2]
 [ 5.3  3.7  1.5  0.2]
 [ 5.   2.3  3.3  1. ]
 [ 6.   3.   4.8  1.8]
 [ 6.5  2.8  4.6  1.5]
 [ 7.   3.2  4.7  1.4]
 [ 5.1  3.8  1.6  0.2]
 [ 4.5  2.3  1.3  0.3]
 [ 7.7  2.8  6.7  2. ]
 [ 4.9  2.5  4.5  1.7]
 [ 4.6  3.1  1.5  0.2]
 [ 5.   3.   1.6  0.2]
 [ 5.1  3.8  1.5  0.3]
 [ 4.8  3.4  1.9  0.2]
 [ 5.2  2.7  3.9  1.4]
 [ 5.2  4.1  1.5  0.1]
 [ 6.3  2.5  4.9  1.5]
 [ 5.2  3.5  1.5  0.2]
 [ 6.8  3.   5.5  2.1]
 [ 6.3  2.5  5.   1.9]
 [ 5.1  2.5  3.   1.1]
 [ 6.8  3.2  5.9  2.3]
 [ 4.9  2.4  3.3  1. ]
 [ 5.   3.6  1.4  0.2]
 [ 7.2  3.   5.8  1.6]
 [ 4.9  3.   1.4  0.2]
 [ 5.   3.4  1.5  0.2]
 [ 4.6  3.6  1.   0.2]
 [ 6.6  2.9  4.6  1.3]
 [ 6.4  3.2  5.3  2.3]
 [ 5

In [30]:
model_hyper2 = {}
model_hyper2['num_classes'] = 3
model_hyper2['input_dim']=4
model_hyper2['num_hidden_layers']=3
model_hyper2['learnrate']=0.0005
layer_size = [None] * model_hyper2['num_hidden_layers']
layer_size[0] = 5
layer_size[1] = 4
layer_size[2] = 5
#layer_size[3] = 6
model_hyper2['layer_sizes'] = layer_size    

In [31]:
iris_nn = NeuralNet_SoftmaxCross(model_hyper2)


(4, 5)
(5,)
__________________
(5, 4)
(4,)
__________________
(4, 5)
(5,)
__________________
(5, 3)
__________________


In [32]:
print trainX[0:10,0:10]

[[ 0.157975 -0.002025 -0.182025 -0.302025]
 [ 0.447975  0.037975  0.297975 -0.142025]
 [ 0.257975 -0.072025  0.167975 -0.182025]
 [ 0.197975  0.047975 -0.172025 -0.302025]
 [ 0.147975 -0.032025 -0.192025 -0.332025]
 [ 0.207975  0.077975 -0.202025 -0.322025]
 [ 0.347975 -0.032025  0.167975 -0.112025]
 [ 0.347975 -0.032025  0.197975 -0.132025]
 [ 0.137975 -0.002025 -0.182025 -0.322025]
 [ 0.187975  0.027975 -0.192025 -0.322025]]


In [33]:
probs_test = iris_nn.forward_test(testX)
print probs_test[:10,:]
pred = np.argmax(probs_test,axis=1)
print pred
print testY,np.count_nonzero(pred == testY),testY.shape
accuracy = np.count_nonzero(pred == testY)/(pred.shape[0]*1.0)
print accuracy

print "Before training ,the accuracy should be similar to guessing probability. ie 1/num_classes"

[[  5.06299249e-01   2.91869976e-03   4.90782051e-01]
 [  5.08722073e-01   4.52826302e-03   4.86749664e-01]
 [  4.25929824e-01   1.19710904e-05   5.74058205e-01]
 [  5.69086592e-01   4.89031341e-05   4.30864505e-01]
 [  4.70336012e-01   1.88864653e-04   5.29475123e-01]
 [  5.40812702e-01   7.31216861e-03   4.51875129e-01]
 [  4.53926435e-01   9.13452074e-05   5.45982220e-01]
 [  3.68617750e-01   3.36938547e-05   6.31348556e-01]
 [  5.03319672e-01   4.40303399e-05   4.96636298e-01]
 [  5.36952554e-01   7.77418563e-05   4.62969704e-01]]
[0 0 2 0 2 0 2 2 0 0 2 0 2 0 0 2 2 2 2 2 2 0 0 2 2 2 0 0 0 0 2 2 2 0 2 2 0
 2 0 2 0 2 2 2 2 2 2 0 2]
[ 2.  2.  0.  0.  1.  2.  1.  0.  0.  0.  0.  1.  1.  1.  2.  1.  1.  1.
  1.  0.  0.  2.  1.  0.  0.  2.  1.  1.  1.  2.  0.  0.  0.  1.  2.  1.
  2.  1.  2.  0.  2.  2.  0.  0.  0.  0.  1.  2.  1.] 6 (49,)
0.122448979592
Before training ,the accuracy should be similar to guessing probability. ie 1/num_classes


In [34]:
batch_size =100
num_train_batch = num_train/batch_size
num_rec = 50000
cost_rec = np.zeros(num_rec,dtype=np.float)
iris_nn.reset_cache()
for i in range(50000):
    th_batch = i%num_train_batch
    Xbatch = trainX[th_batch*batch_size:th_batch*batch_size+batch_size,:]
    Ybatch = trainY[th_batch*batch_size:th_batch*batch_size+batch_size].astype(int)  
    #print Xbatch
    loss_iris,dz_iris = iris_nn.forward_train(Xbatch,Ybatch)
    if i < num_rec:
        cost_rec[i] = loss_iris
    print loss_iris
    iris_nn.backward_train(dz_iris)
    #iris_nn.check_grad_numgrad(Xbatch,Ybatch,0.0001,True)
    iris_nn.Apply_UpdateW()

3.38078189407
3.36340877739
3.34609149017
3.32882981879
3.31162358494
3.29447264589
3.27737689464
3.26033626016
3.24335070773
3.22642023916
3.20954489318
3.19272474576
3.17595991053
3.15925053915
3.14259682175
3.12599898738
3.10945730442
3.09297208112
3.07654366599
3.06017244835
3.04385885879
3.02760336963
3.01140649542
2.99526879336
2.97919086383
2.96317335071
2.94721694189
2.93132236955
2.91549041055
2.89972188673
2.8840176651
2.86837865808
2.85280582362
2.83730016523
2.82186273202
2.80649461853
2.79119696459
2.775970955
2.76081781915
2.74573883052
2.730735306
2.71580860517
2.70096012939
2.68619132073
2.6715036608
2.65689866938
2.64237790289
2.62794295271
2.6135954433
2.59933703017
2.58516939762
2.57109425634
2.55711334081
2.54322840647
2.52944122681
2.51575359013
2.50216729625
2.48868415296
2.47530597234
2.46203456694
2.44887174574
2.43581931002
2.42287904912
2.41005273606
2.39734212308
2.38474893705
2.37227487494
2.35992159913
2.3476907328
2.33558385524
2.3236024973
2.31174813678
2

1.1184334657
1.11787639735
1.11732017646
1.11676480112
1.11621026942
1.11565657946
1.11510372935
1.11455171718
1.11400054107
1.11345019914
1.11290068951
1.1123520103
1.11180415966
1.1112571357
1.11071093658
1.11016556044
1.10962100542
1.10907726969
1.1085343514
1.10799224872
1.10745095982
1.10691048286
1.10637081603
1.10583195751
1.10529390549
1.10475665816
1.10422021371
1.10368457034
1.10314972627
1.1026156797
1.10208242884
1.10154997192
1.10101830716
1.10048743279
1.09995734704
1.09942804814
1.09889953434
1.09837180388
1.09784485502
1.09731868601
1.09679329511
1.09626868059
1.0957448407
1.09522177373
1.09469947794
1.09417795163
1.09365719308
1.09313720057
1.09261797241
1.09209950689
1.09158180231
1.09106485698
1.09054866921
1.09003323732
1.08951855963
1.08900463447
1.08849146015
1.08797903502
1.08746735741
1.08695642566
1.08644623812
1.08593679314
1.08542808907
1.08492012427
1.0844128971
1.08390640593
1.08340064912
1.08289562506
1.08239133212
1.08188776869
1.08138493315
1.08088282389

0.87599768423
0.875739221453
0.875480993214
0.875222999115
0.87496523876
0.874707711753
0.874450417701
0.874193356209
0.873936526885
0.873679929337
0.873423563174
0.873167428007
0.872911523445
0.8726558491
0.872400404586
0.872145189515
0.871890203502
0.871635446162
0.871380917111
0.871126615966
0.870872542345
0.870618695867
0.87036507615
0.870111682816
0.869858515486
0.869605573781
0.869352857325
0.869100365741
0.868848098654
0.86859605569
0.868344236474
0.868092640634
0.867841267798
0.867590117594
0.867339189653
0.867088483605
0.86683799908
0.866587735712
0.866337693133
0.866087870977
0.865838268878
0.865588886472
0.865339723396
0.865090779285
0.864842053779
0.864593546515
0.864345257133
0.864097185274
0.863849330578
0.863601692688
0.863354271246
0.863107065895
0.86286007628
0.862613302046
0.862366742838
0.862120398304
0.861874268091
0.861628351846
0.86138264922
0.861137159861
0.860891883421
0.860646819551
0.860401967902
0.860157328128
0.859912899882
0.859668682818
0.859424676593
0.85

0.723195003616
0.723045627984
0.722896333619
0.722747120436
0.72259798835
0.722448937275
0.722299967127
0.72215107782
0.722002269269
0.72185354139
0.721704894099
0.721556327309
0.721407840938
0.721259434901
0.721111109115
0.720962863494
0.720814697955
0.720666612415
0.720518606791
0.720370680997
0.720222834953
0.720075068573
0.719927381776
0.719779774479
0.719632246598
0.719484798051
0.719337428756
0.71919013863
0.719042927592
0.718895795558
0.718748742448
0.718601768178
0.718454872669
0.718308055837
0.718161317602
0.718014657882
0.717868076597
0.717721573664
0.717575149004
0.717428802535
0.717282534177
0.717136343849
0.716990231471
0.716844196962
0.716698240243
0.716552361233
0.716406559853
0.716260836023
0.716115189662
0.715969620692
0.715824129033
0.715678714606
0.715533377332
0.715388117131
0.715242933925
0.715097827635
0.714952798183
0.714807845489
0.714662969476
0.714518170065
0.714373447178
0.714228800737
0.714084230664
0.713939736881
0.713795319311
0.713650977875
0.713506712497

0.640356748614
0.64024668807
0.640136672769
0.640026702677
0.639916777756
0.639806897973
0.63969706329
0.639587273673
0.639477529086
0.639367829494
0.639258174861
0.639148565152
0.639039000331
0.638929480363
0.638820005213
0.638710574845
0.638601189225
0.638491848318
0.638382552087
0.638273300498
0.638164093517
0.638054931107
0.637945813235
0.637836739864
0.637727710961
0.63761872649
0.637509786417
0.637400890706
0.637292039324
0.637183232235
0.637074469405
0.636965750799
0.636857076382
0.63674844612
0.636639859979
0.636531317924
0.636422819921
0.636314365935
0.636205955931
0.636097589876
0.635989267736
0.635880989476
0.635772755061
0.635664564458
0.635556417633
0.635448314551
0.635340255178
0.635232239481
0.635124267426
0.635016338978
0.634908454103
0.634800612768
0.63469281494
0.634585060583
0.634477349664
0.634369682151
0.634262058008
0.634154477203
0.634046939701
0.633939445469
0.633831994474
0.633724586682
0.63361722206
0.633509900574
0.633402622191
0.633295386878
0.6331881946
0.6

...

0.0928368840482
0.0928359834879
0.0928350829629
0.0928341824732
0.092833282019
0.0928323816
0.0928314812165
0.0928305808682
0.0928296805554
0.0928287802778
0.0928278800357
0.0928269798288
0.0928260796573
0.0928251795212
0.0928242794203
0.0928233793548
0.0928224793247
0.0928215793299
0.0928206793704
0.0928197794462
0.0928188795574
0.0928179797039
0.0928170798857
0.0928161801028
0.0928152803552
0.092814380643
0.0928134809661
0.0928125813244
0.0928116817181
0.0928107821471
0.0928098826114
0.0928089831111
0.092808083646
0.0928071842162
0.0928062848217
0.0928053854625
0.0928044861386
0.09280358685
0.0928026875967
0.0928017883787
0.0928008891959
0.0927999900485
0.0927990909363
0.0927981918594
0.0927972928178
0.0927963938115
0.0927954948405
0.0927945959047
0.0927936970042
0.0927927981389
0.092791899309
0.0927910005143
0.0927901017548
0.0927892030306
0.0927883043417
0.0927874056881
0.0927865070697
0.0927856084865
0.0927847099386
0.092783811426
0.0927829129486
0.0927820145064
0.0927811160995
0.

0.0922343211197
0.0922334440995
0.0922325671134
0.0922316901613
0.0922308132434
0.0922299363595
0.0922290595097
0.092228182694
0.0922273059124
0.0922264291648
0.0922255524513
0.0922246757719
0.0922237991266
0.0922229225153
0.0922220459381
0.092221169395
0.0922202928859
0.0922194164109
0.09221853997
0.0922176635631
0.0922167871903
0.0922159108515
0.0922150345468
0.0922141582761
0.0922132820395
0.0922124058369
0.0922115296684
0.0922106535339
0.0922097774335
0.0922089013671
0.0922080253347
0.0922071493364
0.0922062733721
0.0922053974419
0.0922045215457
0.0922036456835
0.0922027698553
0.0922018940612
0.0922010183011
0.092200142575
0.0921992668829
0.0921983912249
0.0921975156009
0.0921966400108
0.0921957644548
0.0921948889329
0.0921940134449
0.0921931379909
0.0921922625709
0.092191387185
0.092190511833
0.0921896365151
0.0921887612311
0.0921878859812
0.0921870107652
0.0921861355832
0.0921852604352
0.0921843853212
0.0921835102412
0.0921826351952
0.0921817601832
0.0921808852052
0.0921800102611

0.0916354747982
0.0916346209409
0.0916337671164
0.0916329133248
0.0916320595661
0.0916312058402
0.0916303521471
0.091629498487
0.0916286448596
0.0916277912652
0.0916269377036
0.0916260841748
0.0916252306789
0.0916243772158
0.0916235237856
0.0916226703882
0.0916218170236
0.0916209636919
0.091620110393
0.091619257127
0.0916184038938
0.0916175506934
0.0916166975259
0.0916158443912
0.0916149912893
0.0916141382202
0.091613285184
0.0916124321806
0.09161157921
0.0916107262722
0.0916098733672
0.0916090204951
0.0916081676557
0.0916073148492
0.0916064620755
0.0916056093346
0.0916047566265
0.0916039039512
0.0916030513087
0.091602198699
0.0916013461221
0.091600493578
0.0915996410667
0.0915987885882
0.0915979361425
0.0915970837296
0.0915962313494
0.0915953790021
0.0915945266875
0.0915936744058
0.0915928221568
0.0915919699405
0.0915911177571
0.0915902656064
0.0915894134886
0.0915885614035
0.0915877093511
0.0915868573315
0.0915860053447
0.0915851533907
0.0915843014694
0.0915834495809
0.0915825977252


0.0910091725813
0.0910083427085
0.0910075128673
0.0910066830578
0.0910058532798
0.0910050235335
0.0910041938188
0.0910033641357
0.0910025344842
0.0910017048643
0.0910008752761
0.0910000457194
0.0909992161943
0.0909983867009
0.090997557239
0.0909967278088
0.0909958984101
0.090995069043
0.0909942397076
0.0909934104037
0.0909925811314
0.0909917518907
0.0909909226816
0.0909900935041
0.0909892643582
0.0909884352438
0.090987606161
0.0909867771098
0.0909859480902
0.0909851191022
0.0909842901457
0.0909834612208
0.0909826323275
0.0909818034658
0.0909809746356
0.090980145837
0.09097931707
0.0909784883345
0.0909776596306
0.0909768309582
0.0909760023174
0.0909751737082
0.0909743451305
0.0909735165843
0.0909726880698
0.0909718595867
0.0909710311352
0.0909702027153
0.0909693743269
0.0909685459701
0.0909677176448
0.090966889351
0.0909660610888
0.0909652328581
0.090964404659
0.0909635764914
0.0909627483553
0.0909619202507
0.0909610921777
0.0909602641362
0.0909594361263
0.0909586081478
0.0909577802009


0.0903963438962
0.0903955372608
0.0903947306557
0.0903939240812
0.090393117537
0.0903923110233
0.0903915045401
0.0903906980872
0.0903898916648
0.0903890852728
0.0903882789113
0.0903874725802
0.0903866662795
0.0903858600092
0.0903850537693
0.0903842475599
0.0903834413808
0.0903826352322
0.090381829114
0.0903810230262
0.0903802169688
0.0903794109419
0.0903786049453
0.0903777989791
0.0903769930434
0.090376187138
0.0903753812631
0.0903745754185
0.0903737696044
0.0903729638206
0.0903721580673
0.0903713523443
0.0903705466517
0.0903697409895
0.0903689353577
0.0903681297563
0.0903673241853
0.0903665186447
0.0903657131344
0.0903649076546
0.0903641022051
0.090363296786
0.0903624913972
0.0903616860389
0.0903608807109
0.0903600754133
0.090359270146
0.0903584649091
0.0903576597026
0.0903568545265
0.0903560493807
0.0903552442653
0.0903544391803
0.0903536341256
0.0903528291012
0.0903520241073
0.0903512191436
0.0903504142104
0.0903496093075
0.0903488044349
0.0903479995927
0.0903471947808
0.09034638999

In [35]:
print x.dtype,y.dtype,Xbatch.dtype
print Ybatch.dtype

print x.max(),Xbatch.max(),x.min(),Xbatch.min(), Xbatch.mean()
print y.max(),Ybatch.max()
print Xbatch

float64 int64 float64
int64
2.63063889316 0.447975 -1.69001066964 -0.332025 -2.77555756156e-17
2 2
[[ 0.157975 -0.002025 -0.182025 -0.302025]
 [ 0.447975  0.037975  0.297975 -0.142025]
 [ 0.257975 -0.072025  0.167975 -0.182025]
 [ 0.197975  0.047975 -0.172025 -0.302025]
 [ 0.147975 -0.032025 -0.192025 -0.332025]
 [ 0.207975  0.077975 -0.202025 -0.322025]
 [ 0.347975 -0.032025  0.167975 -0.112025]
 [ 0.347975 -0.032025  0.197975 -0.132025]
 [ 0.137975 -0.002025 -0.182025 -0.322025]
 [ 0.187975  0.027975 -0.192025 -0.322025]
 [ 0.157975 -0.112025 -0.012025 -0.242025]
 [ 0.257975 -0.042025  0.137975 -0.162025]
 [ 0.307975 -0.062025  0.117975 -0.192025]
 [ 0.357975 -0.022025  0.127975 -0.202025]
 [ 0.167975  0.037975 -0.182025 -0.322025]
 [ 0.107975 -0.112025 -0.212025 -0.312025]
 [ 0.427975 -0.062025  0.327975 -0.142025]
 [ 0.147975 -0.092025  0.107975 -0.172025]
 [ 0.117975 -0.032025 -0.192025 -0.322025]
 [ 0.157975 -0.042025 -0.182025 -0.322025]
 [ 0.167975  0.037975 -0.192025 -0.312025

In [36]:
probs_test = iris_nn.forward_test(testX)
print probs_test[:10,:]

[[  2.31753059e-13   3.01163839e-03   9.96988362e-01]
 [  2.49282925e-12   1.08693377e-02   9.89130662e-01]
 [  9.75932674e-01   2.40672376e-02   8.87625703e-08]
 [  9.64341825e-01   3.56576830e-02   4.91879716e-07]
 [  1.17111093e-02   9.87229330e-01   1.05956113e-03]
 [  1.22630240e-14   2.08500986e-03   9.97914990e-01]
 [  1.21986319e-01   8.78002212e-01   1.14697007e-05]
 [  9.64742349e-01   3.52573379e-02   3.13066085e-07]
 [  9.49687598e-01   5.03113629e-02   1.03894546e-06]
 [  9.90716632e-01   9.28323329e-03   1.34221728e-07]]


In [37]:
pred = np.argmax(probs_test,axis=1)
print pred
print testY,np.count_nonzero(pred == testY),testY.shape
accuracy = np.count_nonzero(pred == testY)/(pred.shape[0]*1.0)
print accuracy

[2 2 0 0 1 2 1 0 0 0 0 1 1 1 2 1 1 1 1 0 0 2 1 0 0 2 1 1 1 2 0 0 0 1 2 1 2
 1 2 0 2 2 0 0 0 0 1 2 1]
[ 2.  2.  0.  0.  1.  2.  1.  0.  0.  0.  0.  1.  1.  1.  2.  1.  1.  1.
  1.  0.  0.  2.  1.  0.  0.  2.  1.  1.  1.  2.  0.  0.  0.  1.  2.  1.
  2.  1.  2.  0.  2.  2.  0.  0.  0.  0.  1.  2.  1.] 49 (49,)
1.0


In [38]:
import matplotlib.pyplot as plt
plt.plot(cost_rec[0:100])
plt.show()